# Dask SLURMClusters

The MLeRP notebook environment uses Dask SLURMClusters to create a middle ground that has the interactivity of a notebook backed by the power of HPC. You will be provisioned with a CPU based notebook session for your basic analysis and code development. Then, when you're ready to run testsm you will use Dask to submit your python functions to the SLURM queue. 

This enables:
- Flexibility to experiment with your dataset interactively
- Ability to change compute requirements such as RAM, size of GPU, number of processes and so on... without ever leaving the notebook environment
- Elastic scaling of compute
- Efficient utilisation of the hardware
- Releasing of resources when not in use

In [1]:
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster
import dask

# Point Dask to the SLURM to use as it's back end
cluster = SLURMCluster(
    memory="64g", processes=1, cores=8
)

# Scale out to 4 nodes
num_nodes = 4
cluster.scale(num_nodes)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://192.168.0.213:8787/status,
Dashboard: http://192.168.0.213:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.0.213:43565,Workers: 0
Dashboard: http://192.168.0.213:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


Dask will now spin our jobs up in anticipation for work to the scale that you specify.

You can check in on your jobs like you would with any other SLURM job with `squeue`.

In [2]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
              1637     batch Jupyter  mhar0048  R      42:23      1 mlerp-node05
              1638     batch dask-wor mhar0048  R       0:16      1 mlerp-node05
              1639     batch dask-wor mhar0048  R       0:16      1 mlerp-node05
              1640     batch dask-wor mhar0048  R       0:16      1 mlerp-node05
              1641     batch dask-wor mhar0048  R       0:16      1 mlerp-node05


Alternatively, we can use the adapt method, which will let us scale out as we need the compute... and scale back when we're idle letting others use the cluster. 

We reccommend that you use the adapt method while you're actively developing your code so that you don't need to worry about cleaning up after yourself. The scale method can be used when you're ready to run longer tests with higher utilisation.


In [3]:
cluster.adapt(minimum=0, maximum=num_nodes)

In [7]:
# You may need to run this cell a few times while waiting for Dask to clean up
!squeue


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
              1637     batch Jupyter  mhar0048  R      43:01      1 mlerp-node05


Dask has a UI that will let you see how the tasks are being computed.
You won't be able to connect to this with your web browser but VSCode and Jupyter have extensions for you to connect to it. 

Use the loopback address: http://127.0.0.1:8787 (Adjust the port to the one listed when you make the client if needed)

Now let's define a dask array and perform some computation. Dask arrays are parallelised across your workers nodes so they can be greater than the size of one worker's memory. Dask evaluates lazily, retuning 'futures' which record the tasks needed to be completed in the compute graph. They can be computed later for its value. 

Dask also has parallelised implementations of dataframes and collections of objects (called bags). These are written to be as similar as possible to familiar libraries like numpy, pandas and pyspark. You can read more about [arrays](https://docs.dask.org/en/stable/array.html), [dataframes](https://docs.dask.org/en/stable/dataframe.html) and [bags](https://docs.dask.org/en/stable/bag.html) with Dask's documentation.

In [8]:
import dask.array as da
x = da.random.random((1000, 1000, 1000))
x  # Note how the value of the array hasn't been computed yet


dask.array<random_sample, shape=(1000, 1000, 1000), dtype=float64, chunksize=(250, 250, 250), chunktype=numpy.ndarray>

You can check squeue while this is running to see the jobs dynamically spinning up to perform the computation.

In [9]:
x.compute()
x[0][0]

array([[[0.81048276, 0.98448871, 0.1733459 , ..., 0.56455782,
         0.87080788, 0.13560115],
        [0.34637436, 0.87259813, 0.94473364, ..., 0.93145702,
         0.1192914 , 0.29108374],
        [0.41857781, 0.83517549, 0.08113376, ..., 0.85804457,
         0.37388268, 0.37722865],
        ...,
        [0.46583782, 0.61815175, 0.09393558, ..., 0.66240159,
         0.49874618, 0.81926554],
        [0.24346786, 0.60615724, 0.99073193, ..., 0.06461803,
         0.00183846, 0.67093819],
        [0.08790386, 0.83009404, 0.12218377, ..., 0.06221952,
         0.02795862, 0.71642665]],

       [[0.87904067, 0.08657706, 0.54566209, ..., 0.88747936,
         0.67862782, 0.06280928],
        [0.60127705, 0.16065682, 0.20523986, ..., 0.37973246,
         0.63451103, 0.76550924],
        [0.40090817, 0.03877228, 0.40892043, ..., 0.28284783,
         0.08165901, 0.62547838],
        ...,
        [0.91598835, 0.60142513, 0.58315254, ..., 0.4106674 ,
         0.39268422, 0.0359192 ],
        [0.4

We can also accelerate dask arrays with GPUs using cupy. There is similar support for accelerating dask dataframes with CuDF.

In [10]:
dask.config.set({"array.backend": "cupy"})
y = da.random.random((1000, 1000, 1000))
y.compute()
y[0][0]

array([[[3.96333169e-03, 3.75975260e-01, 3.56959376e-01, ...,
         4.50280915e-01, 2.35157391e-01, 7.94328044e-01],
        [7.46712819e-01, 3.80351024e-02, 6.20411425e-01, ...,
         4.25249481e-01, 1.79268928e-01, 4.18742215e-01],
        [6.86661418e-01, 2.68955439e-01, 1.35690349e-01, ...,
         6.07946338e-01, 5.99222969e-01, 9.05284417e-01],
        ...,
        [9.76543019e-01, 6.83623666e-01, 7.72140730e-02, ...,
         5.01167957e-01, 9.58515852e-01, 5.07214775e-01],
        [2.40601659e-01, 8.18106652e-01, 2.82227562e-01, ...,
         4.40147690e-01, 2.40043011e-01, 7.48213851e-01],
        [6.46267786e-01, 2.13913358e-01, 2.05269982e-01, ...,
         8.90589693e-01, 5.19737573e-01, 9.10289392e-01]],

       [[1.72304898e-01, 5.05337770e-01, 1.87073213e-01, ...,
         6.76683193e-01, 1.69135886e-01, 2.43992433e-01],
        [3.70932825e-01, 8.84938115e-01, 8.77260345e-01, ...,
         4.57830983e-01, 5.27520536e-01, 2.01397156e-01],
        [9.09455520e-01, 

Finally, we can shut down the SLURMCluster now that we're done with it.

In [11]:
# Shut down the cluster
client.shutdown()
